# Testing storage-loading bay transport operations
### 1 Traffic Generator
### 2 Terminal Design
### 3 Simulation
- 3.1 Simulation Parameters
- 3.2 Storage - loading conveyor development
- 3.3 Test case

In [1]:
import copy
import plotly
import numpy as np
import pandas as pd

import plotly.tools as tls 
import plotly.plotly as py  
import plotly.graph_objs as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline

import terminal_optimization.visualisation               as visualisation
import terminal_optimization.forecast                    as forecast
import terminal_optimization.investment_decisions        as invest
import terminal_optimization.infrastructure              as infra
import terminal_optimization.financial_analysis          as financial
import terminal_optimization.initial_terminal            as initial
import terminal_optimization.investment_decisions_future as future_invest

# Log in to Plotly servers
#plotly.tools.set_credentials_file(username='jorisneuman', api_key='zButeTrlr5xVETcyvazd')
#plotly.tools.set_credentials_file(username='wijzermans', api_key='FKGDvSah3z5WCNREBZEq')
plotly.tools.set_credentials_file(username='wijnandijzermans', api_key='xeDEwwpCK3aLLR4TIrM9')

colors = ['rgba(55, 83, 109, 1)', 'rgba(55, 83, 109, 0.8)', 'rgba(55, 83, 109, 0.5)',
          'rgb(129, 180, 179, 1)', 'rgb(79, 127, 127, 1)', 'rgb(110,110,110)'] 

## 1 Traffic Generator

In [2]:
def traffic_generator(nr_generations, start_year, simulation_window):
    
    ###################################################################################################
    # Traffic projections on which estimate is based
    ###################################################################################################

    # Import commodities from package
    maize   = forecast.bulk_commodities(**forecast.maize_data)
    soybean = forecast.bulk_commodities(**forecast.maize_data)
    wheat   = forecast.bulk_commodities(**forecast.wheat_data)

    # Maize - demand
    maize.historic = [750000,750000,750000,750000,750000]
    maize.rate     = 1.0636 
    maize.sigma    = 0.0374

    # If defined number of traffic generations is even, one iteration is added in order to be able to identify the median
    if nr_generations % 2 == 0:
        nr_generations = nr_generations + 1 

    # Create traffic projections
    traffic_projections = []
    for iterations in range(nr_generations):
        maize.random_scenario(start_year, simulation_window, maize.historic, maize.rate, maize.sigma)
        traffic_projections.append(maize.demand)
    
    # Calculate the median traffic projection (i.e. traffic scenario)
    traffic_scenario = []
    
    for i in range (len(maize.historic)):
        traffic_scenario.append(
            maize.historic[i])
        
    for year in range (len(maize.historic), len(maize.historic) + simulation_window):
        throughputs = []
        for i in range (len(traffic_projections)):
            throughputs.append(
                traffic_projections[i][year])
        traffic_scenario.append(
            np.median(throughputs))
        
    # Define commodity demand     
    maize.demand   = traffic_scenario 
    soybean.demand = [0]*len(maize.demand)
    wheat.demand   = [0]*len(maize.demand)
    commodities    = [maize, soybean, wheat]
    
    # Override commodity demand for this example
    maize.demand   = [750000,750000,750000,750000,750000, 750000, 750000, 750000, 750000, 750000, 1600000, 1600000, 1600000, 1600000, 1600000] 
    soybean.demand = [0]*len(maize.demand)
    wheat.demand   = [0]*len(maize.demand)
    commodities    = [maize, soybean, wheat]
    
    # Import vessels from package
    handysize = forecast.vessel(**forecast.handysize_data)
    handymax  = forecast.vessel(**forecast.handymax_data)
    panamax   = forecast.vessel(**forecast.panamax_data)
    vessels = [handysize, handymax, panamax]
    
    # Calculate yearly calls
    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)

    # Plot forecast
    fig = visualisation.scenario(traffic_projections, commodities)
    
    return vessels, commodities, fig, traffic_projections, traffic_scenario

## 2 Temporal Terminal Design

In [3]:
def design(chosen_method, terminal, vessels, commodities, start_year, simulation_window):
    
    for i in range (start_year, start_year + simulation_window): 
        
        year = i 
        timestep = year - start_year

        ##################################################################
        # Investment Decisions (current performance method)
        ##################################################################     
        
        if chosen_method == 'Current performance method':
            
            # Berths and cranes
            terminal.berths, terminal.cranes = invest.berth_invest_decision(terminal.berths, terminal.cranes, commodities, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)
            for i in range (len(terminal.berths)):
                terminal.berths[i].crane_type = 'Mobile cranes' 
                
            # Quay
            terminal.quays = invest.quay_invest_decision(terminal.quays, terminal.berths, vessels, year, timestep)

            # Storage
            storage_type = 'Silos'
            terminal.storage = invest.storage_invest_decision(terminal.storage, terminal.dwell_time, vessels, storage_type, commodities, year, timestep, operational_hours)

            # Loading stations
            terminal.stations, terminal.trains = invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

            # Conveyors
            terminal.quay_conveyors = invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.berths, terminal.cranes, commodities, year, timestep, operational_hours)
            terminal.hinterland_conveyors = invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, commodities,  year, timestep, operational_hours)

        ##################################################################
        # Investment Decisions (Perfect foresight and forecast method)
        ################################################################## 
        
        if chosen_method == 'Perfect foresight method' or chosen_method == 'Forecast based method':
            
            # Create forecast and accompanying vessel calcs
            commodities = forecast.forecaster(chosen_method, 'Linear', commodities, forecast_window, hindcast_window, timestep)
            vessels = forecast.forecast_call_calc(vessels, commodities, simulation_window)
            
            # Berths and cranes
            terminal.berths, terminal.cranes = future_invest.berth_invest_decision(terminal.berths, terminal.cranes, commodities, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)

            # Quay
            terminal.quays = future_invest.quay_invest_decision(terminal.quays, terminal.berths, vessels, year, timestep)

            # Storage
            storage_type = 'Silos'
            terminal.storage = future_invest.storage_invest_decision(terminal.storage, terminal.dwell_time, vessels, storage_type, commodities, year, timestep, operational_hours)

            # Loading stations
            terminal.stations, terminal.trains = future_invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

            # Conveyors
            terminal.quay_conveyors = future_invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.berths, terminal.cranes, commodities, year, timestep, operational_hours)
            terminal.hinterland_conveyors = future_invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, commodities, year, timestep, operational_hours)      
        
        ##################################################################
        # Business logic
        ################################################################## 
        
        # Terminal throughput
        terminal = financial.throughput_calc(terminal, commodities, vessels, terminal.trains, operational_hours, timestep, year)
        # Revenue
        terminal.revenues = financial.revenue_calc(terminal.revenues, terminal.throughputs, commodities, year, timestep)       
        # Capex
        terminal.capex = financial.capex_calc(terminal, year, timestep)
        # Labour costs
        terminal.labour = financial.labour_calc(terminal, year, timestep, operational_hours)
        # Maintenance costs
        terminal.maintenance = financial.maintenance_calc(terminal, year, timestep)
        # Energy costs
        terminal.energy = financial.energy_calc(terminal, year, operational_hours, timestep)
        # Insurance costs
        terminal.insurance = financial.insurance_calc(terminal, year, timestep)
        # Lease costs 
        terminal.lease = financial.lease_calc(terminal, year,timestep)
        # Demurrage costs
        terminal.demurrage = financial.demurrage_calc(terminal.demurrage, terminal.berths, terminal.cranes, commodities, vessels, operational_hours, timestep, year)
        # Residual value calculations 
        terminal.residuals = financial.residual_calc(terminal, year, timestep)
        # Profits
        terminal.profits = financial.profit_calc(terminal, simulation_window, timestep, year, start_year)
        # Opex
        terminal.opex = financial.opex_calc(terminal, year, timestep)  
        
    #WACC depreciated profits
    terminal.WACC_cashflows = financial.WACC_calc(terminal.project_WACC, terminal.profits, terminal.revenues, terminal.capex, terminal.opex, simulation_window, start_year)
    
    # Combine all cashflows
    terminal.cashflows = financial.cashflow_calc(terminal, simulation_window, start_year) 
    
    #NPV 
    terminal.NPV = financial.NPV_calc(terminal.WACC_cashflows)
        
    return terminal

# 3 Simulation

## 3.1 Simulation parameters

In [4]:
# Simulation parameters
start_year        = 2018   # start year of simulation
simulation_window = 10     # forecast 20 years
end_year          = start_year + simulation_window - 1
operational_hours = 5840   # operational hours per year (16 hours per day 365 days a year)
project_WACC      = 0.095  # Applied project weighted average cost of capital
nr_generations    = 1      # Number of traffic projections generated

# In case of future based methods:
hindcast_window = 5  # Number of years of materialized traffic volumes that are included in assessment
forecast_window = 2  # Number of years of forecasted traffic volumes

# Choose design method
#chosen_method = 'Perfect foresight method'
chosen_method = 'Current performance method'
#chosen_method = 'Forecast based method'

# Performance triggers (static values)
triggers = []
triggers.append([0.30, 'Acceptable waiting vessel time as factor of service time'])
triggers.append([18  , 'Dwell time of cargo in storage, presented in days'])         
triggers.append([0.50, 'Acceptable waiting train time as factor of service time'])

# Make traffic projections
vessels, commodities, fig, traffic_projections, traffic_scenario = traffic_generator(nr_generations, start_year, simulation_window)

# Override traffic projection for test purposes
commodities[0].demand = [1200000,1200000,1200000,1200000,1200000, 1200000, 1200000, 1200000, 1200000, 1200000, 2400000, 2400000, 2400000, 2400000, 2400000]
commodities[0].utilization = 1.00
maize = commodities[0] 
vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)

# Run simulation
terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

# Visualize traffic projections
fig = visualisation.traffic_development(commodities)
py.iplot(fig, filename='Traffic projections')

## 3.2 Storage - loading conveyor development

In [5]:
# Visualize berth development
df1 = terminal.hinterland_conveyors[0].info
df2 = terminal.stations[0].info

trace1 = {
  'x': df1['Year'],
  'y': df1['Capacity'],
  'name': 'Conveying<br>capacity',
  'type': 'bar',
  'marker': dict(color=colors[0])
};

trace2 = {
  'x': df2['Year'],
  'y': df2['Loading capacity'] * df2['Nr of stations'],
  'name': 'Peak loading<br>capacity',
  'type': 'bar',
  'marker': dict(color='rgba(170, 170, 170, 0.6)')
};

trace3 = go.Scatter(
    name='Traffic volume',
    x = maize.years,
    y = maize.demand,
    yaxis='y2',
    mode='lines',
    line=dict(
        color='rgb(214, 39, 40)', 
        width=2,
        shape='hv'
    )
)

data = [trace1, trace2, trace3];
layout = {
    'xaxis': dict(
            range = [2019.5,2027.5],
            tick0=0,
            dtick=1,
            tickangle=315),
    'yaxis': dict(title='Capacity (t/h)',
                  range = [10, 2100]),
    'yaxis2':dict(
                title='Traffic volume (t/year)',
                overlaying='y',
                side='right',
                showgrid=False,
                range = [1000, 3100000]
                ),
    'title': 'Transport capacity development',
    'legend': dict(x=1.1,
                 y=1),
    'bargap': 0.18,
    'bargroupgap' :0.1,
    };

annotations = []

annotations.append(dict(
                        x=1,
                        y=-0.15,
                        showarrow=False,
                        text='Source: URL or QR code',
                        xref='paper',
                        yref='paper'))

for values in [2021, 2023, 2025.5]:
    if values == 2021:
        text = 'Stage 1'
    if values == 2023:
        text = 'Stage 2'
    if values == 2025.5:
        text = 'Stage 3'
    annotations.append(dict(
                        x=values,
                        y=1.05,
                        showarrow=False,
                        text=text,
                        yref='paper',
                        font=dict(
                                size=14)))
    
annotations.append(dict(
                        x=2021,
                        y=0.45,
                        showarrow=False,
                        text='Conveying capacity: '+str(int(df1['Capacity'].values[1]))+' t/h',
                        yref='paper',
                        font=dict(
                                size=14,
                                color=colors[0])))

annotations.append(dict(
                        x=2021,
                        y=0.50,
                        showarrow=False,
                        text='Loading capacity: '+str(int(df1['Hinterland capacity demand'].values[0]))+' t/h',
                        yref='paper',
                        font=dict(
                                size=14,
                                color='rgba(170, 170, 170, 0.9)')))

annotations.append(dict(
                        x=2025.5,
                        y=0.89,
                        showarrow=False,
                        text='Conveying capacity: '+str(int(df1['Capacity'].values[-1]))+' t/h',
                        yref='paper',
                        font=dict(
                                size=14,
                                color=colors[0])))

annotations.append(dict(
                        x=2025.5,
                        y=0.94,
                        showarrow=False,
                        text='Loading capacity: '+str(int(df1['Hinterland capacity demand'].values[-1]))+' t/h',
                        yref='paper',
                        font=dict(
                                size=14,
                                color='rgba(170, 170, 170, 0.9)')))

shapes = []

for values in [2022.5, 2023.5]:
    shapes.append(dict(
                        type='line',
                        x0=values,
                        y0=0,
                        x1=values,
                        y1=1.05,
                        yref='paper',
                        line=dict(
                            color='rgb(60, 60, 60)',
                            width=3,
                            dash='dashdot')))

layout['annotations'] = annotations
layout['shapes'] = shapes

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Asset development')

## 3. Test Case

In [9]:
def conveyor_test(df, year):
    
    timestep = year - 2019
    loading_capacity = df['Hinterland capacity demand'].values[timestep]
    conveyor_capacity = df['Capacity'].values[timestep]
    
    print('Results for ' + str(year) + ':')
    print('Cumulative loading capacity:', loading_capacity)
    print('Conveying capacity:', conveyor_capacity)
    print()
    
    return

In [10]:
df = terminal.hinterland_conveyors[0].info

# Testing in 2022
conveyor_test(df, 2022)

# Testing in 2023
conveyor_test(df, 2023)

# Testing in 2024
conveyor_test(df, 2024)

Results for 2022:
Cumulative loading capacity: 800.0
Conveying capacity: 800.0

Results for 2023:
Cumulative loading capacity: 800.0
Conveying capacity: 800.0

Results for 2024:
Cumulative loading capacity: 1600.0
Conveying capacity: 1600.0

